In [2]:
import pandas as pd
import numpy as np
import re

In [39]:
pd.set_option('display.min_rows',0) # less likely helpful except specific circumstance
pd.set_option('display.max_rows',None) 

##### Check scraped 700 pages

In [ ]:
df = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/kicktraq/kicktraq/kicktraq.csv')

In [ ]:
df.shape[0]/15 #15 projects per page


In [ ]:
df.shape[0]

###### Rescraped to include try except=None on backers

In [ ]:
df2 = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/kicktraq/kicktraq/kicktraq2.csv')

In [ ]:
df2.shape[0]/15 # check if 700 pages

##### Check to see if any backers null

In [ ]:
df2.loc[df2['backers'].isnull()] #no nulls

##### Check to see if two scrapes returned same number rows

In [ ]:
df2.shape[0] == df.shape[0]

# Transformation

In [3]:
# Using df2 reassign as dfn for newly scraped kicktraq - to distinguish between older kaggle dataset (dfo)
dfn = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/kicktraq/kicktraq/kicktraq2.csv')

In [4]:
dfn.sample(5)

,backers,category,currency,description,goal,main_category,names,pledged,proj_details,status
1343,4,NaN,€,Luthier laboratory for Italian handmade instru...,22000.0,crafts,Wabi-Sabi Guitars - Handcrafted in Italy,13.0,"\n\t\t\t\t\tBackers: 4,\n\t\t\t\t\tFunding: €1...",Closed
6776,2,games|playing cards,$,This card game will keep you and your family ...,4400.0,games,Stay at home game.,2.0,"\n\t\t\t\t\tBackers: 2,\n\t\t\t\t\tFunding: $2...",Closed
3054,65,NaN,$,An enamel pin set featuring nature's beauty.,400.0,art,Lily Pond- Enamel Pin Set,2371.0,"\n\t\t\t\t\tBackers: 65,\n\t\t\t\t\tFunding: $...",Closed
5750,9,publishing|fiction,$,The fourth standalone novel by D. L. Silvertho...,2000.0,publishing,Shotgun,191.0,"\n\t\t\t\t\tBackers: 9,\n\t\t\t\t\tFunding: $1...",Closed
471,59,games|tabletop games,$,A highly engaging remote team building activit...,200.0,games,Project U - An Escape Room Game for remote teams,1211.0,"\n\t\t\t\t\tBackers: 59,\n\t\t\t\t\tFunding: $...",Closed


In [5]:
# dropping some columns, will get from proj_details with more accuracy
dfn = dfn.drop(['currency', 'goal', 'pledged'], axis=1)

### Splitting category

In [6]:
# split on '|' make new dfn_cat
dfn_cat = dfn['category'].str.split('|',expand=True) # merge to data frame

In [7]:
# concat dfn and dfn_cat, drop 0 (name of the first component of split), rename second new column 'sub_category'
dfn = pd.concat([dfn, dfn_cat], axis=1).drop(0,1).rename(columns = {1:'sub_category'})

In [8]:
dfn.sample(5)

,backers,category,description,main_category,names,proj_details,status,sub_category
9819,20,NaN,"Grim Lemons is creating stickers, pins, plushi...",art,Grim Lemons Kickoff,"\n\t\t\t\t\tBackers: 20,\n\t\t\t\t\tFunding: $...",Closed,NaN
10001,3,fashion|apparel,Delivering luxury through high quality and end...,fashion,Instinct Copenhagen: Statement Pieces & Access...,"\n\t\t\t\t\tBackers: 3,\n\t\t\t\t\tFunding: kr...",Closed,apparel
3185,1419,games|tabletop games,Three new Undead teams for Fantasy Football. M...,games,"Vampires, Egyptians and Necromancers for Fanta...","\n\t\t\t\t\tBackers: 1419,\n\t\t\t\t\tFunding:...",Closed,tabletop games
1075,8,technology|apps,A delivery App that allows you to purchase dri...,technology,Skygoody- Buy your friends drinks on airlines ...,"\n\t\t\t\t\tBackers: 8,\n\t\t\t\t\tFunding: $1...",Closed,apps
6583,1978,design|product design,Amazing last generation of silicone Brush for ...,design,SCRUBBIO® 8-in-1: Eco Multifunctional Magic Si...,"\n\t\t\t\t\tBackers: 1978,\n\t\t\t\t\tFunding:...",Closed,product design


##### Extracting date portion of proj_details into 'campaign_dates' col - there are no missing values 

In [9]:
### KEEP THIS! THIS GETS campaign dates as a chunck will play with to get datetimes
dfn['campaign_dates'] = [re.findall('Campaign (Dates: .+)', el)[0] for el in dfn['proj_details']]    

##### 'launched' col from campaign_dates

In [10]:
# creates list of tuples with Month, day , year
start_date = [re.findall('Dates: (\D*) (\d*).+\((\d+)\)', el)[0] for el in dfn['campaign_dates']]

In [11]:
# join tuple elements into string, now list of strings
start_date = [' '.join(date) for date in start_date]

In [12]:
# create column 'launched' with datetime object from list
dfn['launched']= [pd.datetime.strptime(s, '%B %d %Y') for s in start_date]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [13]:
dfn.head(5)

,backers,category,description,main_category,names,proj_details,status,sub_category,campaign_dates,launched
0,10,food|drinks,A do it yourself boba tea kit that allows you ...,food,Boba Era: Boba From Home,"\n\t\t\t\t\tBackers: 10,\n\t\t\t\t\tFunding: $...",Cancelled,drinks,"Dates: June 3rd -> August 2nd (2020),",2020-06-03
1,1,fashion|apparel,Sophisticated Mexican old school style apparel.,fashion,Mr Cochino Apparel,"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: $1...",Cancelled,apparel,"Dates: June 26th -> August 2nd (2020),",2020-06-26
2,1,fashion|apparel,RotenStein Clothing Co. Una nueva marca de rop...,fashion,RotenStein Clothing Co. (Arte en tu ropa),"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: MX...",Cancelled,apparel,"Dates: June 3rd -> August 2nd (2020),",2020-06-03
3,1,games|video games,"Z-City, A Zombie based Survival-Battle Royale ...",games,Z-City Survival-Battle Royale,"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: $1...",Cancelled,video games,"Dates: July 3rd -> August 2nd (2020),",2020-07-03
4,0,comics|graphic novels,A Samurai Horror Graphic Novel,comics,Death Kanji,"\n\t\t\t\t\tBackers: 0,\n\t\t\t\t\tFunding: $0...",Cancelled,graphic novels,"Dates: July 3rd -> August 2nd (2020),",2020-07-03


##### 'deadline' same as 'launched' above

In [14]:
end_date = [re.findall('-> (\D+) (\d+).+\((\d+)\)', el)[0] for el in dfn['campaign_dates']]

In [15]:
end_date = [' '.join(date) for date in end_date]

In [16]:
# drop campaign dates
dfn['deadline']= [pd.datetime.strptime(s, '%B %d %Y') for s in end_date]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [17]:
dfn.sample(5)

,backers,category,description,main_category,names,proj_details,status,sub_category,campaign_dates,launched,deadline
2135,4,music|kids,Children's songs,music,The Godino Show,"\n\t\t\t\t\tBackers: 4,\n\t\t\t\t\tFunding: $1...",Closed,kids,"Dates: April 9th -> April 24th (2020),",2020-04-09,2020-04-24
1080,34,games|video games,"An old-school real time strategy, inspired by ...",games,EMPEROR: The Battle Of Dynasties,"\n\t\t\t\t\tBackers: 34,\n\t\t\t\t\tFunding: $...",Closed,video games,"Dates: March 15th -> April 12th (2020),",2020-03-15,2020-04-12
8115,61,comics|comic books,art book,comics,BADGIRL SKETCHBOOK VOL.19,"\n\t\t\t\t\tBackers: 61,\n\t\t\t\t\tFunding: $...",Closed,comic books,"Dates: June 25th -> July 1st (2020),",2020-06-25,2020-07-01
5554,180,games|playing cards,WHITE EDITION.,games,WARRIOR PLAYING CARDS by RJ,"\n\t\t\t\t\tBackers: 180,\n\t\t\t\t\tFunding: ...",Closed,playing cards,"Dates: May 5th -> June 4th (2020),",2020-05-05,2020-06-04
713,43,games|tabletop games,Themed Sets in unique 3D-design boxes combined...,games,SPIRIT OF ADVENTURE | MULTICOLOR GEAR DICE,"\n\t\t\t\t\tBackers: 43,\n\t\t\t\t\tFunding: $...",Closed,tabletop games,"Dates: March 9th -> April 9th (2020),",2020-03-09,2020-04-09


##### Extracting funding portion of proj_details into 'funding' col also deals with missing values

In [18]:
##### KEEP THIS HOW WE GOT THE FUNDING STRING EVEN IF MISSING
dfn['funding'] = [None if re.findall('Funding: (.*)', el)==[] else re.findall('Funding: (.*)', el)[0] for el in dfn['proj_details']] 

##### Spliting funding into three columns: currency, pledged, goal

In [20]:
dfn['funding'] = dfn['funding'].str.replace(',','')

In [21]:
# turn funding into tuple (currency, pledged, goal) take into account None types in list comprehension
dfn['funding'] = [None if el==None else re.findall('(\D*)(\d*) of \D*(\d*) \(', el)[0] for el in dfn['funding']] 

In [22]:
# splitting tuple into columns in new dfn_fund
dfn_fund = dfn['funding'].apply(pd.Series)

In [23]:
# renaming columns
dfn_fund.columns=['currency_sym', 'pledged', 'goal']

In [24]:
dfn_fund.head(5)

,currency_sym,pledged,goal
0,$,401,3000
1,$,1,16578
2,MX$,19,73000
3,$,1,5000
4,$,0,7500


In [25]:
# concat dfn and dfn_fund and drop unnecessary columns
dfn = pd.concat([dfn, dfn_fund], axis = 1).drop(['proj_details', 'category', 'campaign_dates', 'funding'], axis = 1)

In [27]:
dfn.head(5)

,backers,description,main_category,names,status,sub_category,launched,deadline,currency_sym,pledged,goal
0,10,A do it yourself boba tea kit that allows you ...,food,Boba Era: Boba From Home,Cancelled,drinks,2020-06-03,2020-08-02,$,401,3000
1,1,Sophisticated Mexican old school style apparel.,fashion,Mr Cochino Apparel,Cancelled,apparel,2020-06-26,2020-08-02,$,1,16578
2,1,RotenStein Clothing Co. Una nueva marca de rop...,fashion,RotenStein Clothing Co. (Arte en tu ropa),Cancelled,apparel,2020-06-03,2020-08-02,MX$,19,73000
3,1,"Z-City, A Zombie based Survival-Battle Royale ...",games,Z-City Survival-Battle Royale,Cancelled,video games,2020-07-03,2020-08-02,$,1,5000
4,0,A Samurai Horror Graphic Novel,comics,Death Kanji,Cancelled,graphic novels,2020-07-03,2020-08-02,$,0,7500


### Currency - adding currency code and USD exchange rate

In [29]:
dfn.loc[dfn['currency_sym']=='kr']

,backers,description,main_category,names,status,sub_category,launched,deadline,currency_sym,pledged,goal
49,3,A book about dimension mapping and meetings wi...,publishing,Clockwork Elves and DMT mapping,Cancelled,NaN,2020-07-08,2020-08-07,kr,289,16700
209,14,A small series of elegant holders in a Scandin...,design,Edge Holders,Closed,product design,2020-06-18,2020-07-18,kr,4862,60000
266,13,We are recording our debut album.,music,Communist Sex: Lada Riva,Closed,indie rock,2020-02-05,2020-04-05,kr,3089,3000
327,10,Reborn project of Swedish 80's metal band Ivory.,music,Ivory Reborn - Swedish 80's metal. Part 1 – di...,Closed,metal,2020-02-25,2020-04-05,kr,3879,75000
338,1,The world's first Social Media that allows you...,technology,THE DABO APP: Challenge the world,Cancelled,apps,2020-02-26,2020-04-06,kr,8,800000
...,...,...,...,...,...,...,...,...,...,...,...
10297,841,Epic narrative adventure board game for 1-4 pl...,games,"Unlikely Heroes: Vikings, Cowboys & Pharaohs",Closed,tabletop games,2020-06-23,2020-07-23,kr,367794,10000
10398,4,I have created a luxury inspired watch design ...,fashion,"Marshall's Ace - ""Eye Ticking"" Watch Brand Fro...",Cancelled,accessories,2020-06-22,2020-07-24,kr,5105,170000
10402,2,A Visual Music Album within the Pop Genre to e...,music,'The first breath of a Rainbow (A Visual Music...,Cancelled,pop,2020-05-25,2020-07-24,kr,259,170000
10415,82,The 2 in 1 vase that brings security and safet...,design,The FireVase,Cancelled,NaN,2020-05-27,2020-07-26,kr,45042,249355


In [28]:
dfn['currency_sym'].value_counts()

$      6661
€      1560
£      1446
MX$     292
kr      232
HK$     220
¥        71
Name: currency_sym, dtype: int64

### Splitting category

In [ ]:
#### KEEP
dfn_cat = dfn['category'].str.split('|',expand=True) #this one and merge to data frame

###### concat dfn and dfn_cat, drop 0 (name of the first component of split), rename second new column 'sub_category'

In [ ]:
#### KEEP
dfn = pd.concat([dfn, dfn_cat], axis=1).drop(0,1).rename(columns = {1:'sub_category'})

In [ ]:


dfn.sample(5)

In [ ]:
# dfn['main_category'] = [el.split('|')[0] for el in dfn['category']] # would work if had no missing
# dfn['sub_category'] = [el.split('|')[1] for el in dfn['category']]

In [ ]:
dfn.columns

In [ ]:
# test_list = ['\n\t\t\t\t\tBackers: 10,\n\t\t\t\t\tFunding: $401 of $3,000 (,13% funded,),\n\t\t\t\t\tCampaign Dates: June 3rd -> August 2nd (2020),\n\t\t\t\t\tClosed: Sunday August 2nd, 2020,\n\t\t\t\t',
#  '\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tCampaign Dates: June 26th -> August 2nd (2020),\n\t\t\t\t\tClosed: Sunday August 2nd, 2020,\n\t\t\t\t',
#  '\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: MX$19 of MX$73,000 (,0% funded,),\n\t\t\t\t\tCampaign Dates: June 3rd -> August 2nd (2020),\n\t\t\t\t\tClosed: Sunday August 2nd, 2020,\n\t\t\t\t',]

##### Changing currency

In [ ]:
dfn['currency'].value_counts() # see what currency we have

##### creating currency exchange data frame

In [ ]:
symb_dict = {'currency':['$','€','£', '¥'], 'currency_code':['USD', 'EUR', 'GPB', 'JPY'], 'exchange_rt_to_USD': [1, 1.16, 1.28, 0.0094]}
df_cur_sym = pd.DataFrame(symb_dict)
df_cur_sym


##### merge dfn with currency exchange

In [ ]:
dfn = pd.merge(dfn, df_cur_sym, how='left', on='currency')

In [ ]:
dfn.sample(5)

#### updating status column to include fail and success, percent funded, usd pledged, usd goal

In [ ]:
df2_sort_names=df2.sort_values('names')['names']
df2_sort_names=df2_sort_names.dropna(axis=0, how='any')
df2_sort_names.shape[0]

In [ ]:
merged_names = pd.merge(df_sort_names, df2_sort_names, how = 'outer', on='names')
merged_names.shape[0]/15

In [ ]:
merged_names.loc[merged_names]

In [ ]:
here = ['! EMPOWER MY POTENTIAL - DEVOUR MY MENTAL !',
 '" " -> Quotidian - a poetry debut collection',]
there = ['! EMPOWER MY POTENTIAL - DEVOUR MY MENTAL !',
 '" " -> Quotidian - a poetry debut collection',]
here == there

In [ ]:
df2_sort_names == df_sort_names

In [ ]:
# month_dict = {'January':'01', 'February':'02', 'March':'03', 'April':'04', 'May':'05', 'June':'06', 'July':'07', 'August':'08', 'September':'09', 'October':'10', 'November':'11', 'December':'12'}

In [ ]:
# category = ['/categories/main/']
# main_category = ''
# if len(category) == 1:
#     main_category = category[0].strip('/').replace('categories/','')
#     category = None
# else:
#     main_category = category[0].strip('/').replace('categories/','')
#     category = category[1].strip('/').replace('categories/','').replace('/','|')
# print(main_category)
# print(category)

In [30]:
df = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/data/Web_Scraping_Proj/ks-projects-201801.csv')
df.sample(20)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
242273,302203119,The Searzall,Food,Food,USD,2013-12-27,80000.0,2013-11-29 06:01:06,226425.00,successful,2811,US,226425.00,226425.00,80000.00
47291,1240888919,ABODE: Thematic Gaming Accessories,Tabletop Games,Games,USD,2014-11-09,1000.0,2014-10-10 18:07:01,6184.00,successful,113,US,6184.00,6184.00,1000.00
65419,1332725228,Game Boy Geek - Season 5 - 2017,Tabletop Games,Games,USD,2017-03-15,20000.0,2017-02-13 08:00:25,24031.00,successful,472,US,12573.00,24031.00,20000.00
136818,1694774648,SHA'CUP - The Innovative All-in-one Cocktail's...,Product Design,Design,HKD,2017-11-25,150000.0,2017-09-26 15:40:42,131604.00,failed,256,HK,755.39,16845.10,19199.75
138558,1703626794,Aroma-Spa,Crafts,Crafts,USD,2015-05-20,1500.0,2015-04-20 18:25:41,0.00,failed,0,US,0.00,0.00,1500.00
175047,1890249640,Connected Hearts Journal: Fun Memory Book. Sma...,Product Design,Design,USD,2016-11-11,35000.0,2016-10-12 13:25:56,36997.00,successful,542,US,5593.00,36997.00,35000.00
120197,1610465493,Corsa Cleat: A Modern Bicycle Cleat for Vintag...,Product Design,Design,USD,2014-07-18,10500.0,2014-06-17 17:45:45,814.00,failed,14,US,814.00,814.00,10500.00
8989,1045401555,"""Dreamin' of You"" needs support with funding. ...",Film & Video,Film & Video,USD,2012-07-04,300000.0,2012-05-17 02:08:34,100.00,failed,3,US,100.00,100.00,300000.00
229250,235546304,JiHAE's new album - Illusion of You,Indie Rock,Music,USD,2013-10-06,127000.0,2013-09-03 22:22:46,128425.00,successful,278,US,128425.00,128425.00,127000.00
333843,770596738,2020 Vision Ultra High End 4k Action Camera Sp...,Camera Equipment,Technology,GBP,2017-05-20,25000.0,2017-04-20 22:48:37,1968.00,failed,10,GB,2652.84,2527.09,32102.32


In [53]:
df.columns.tolist()

['ID',
 'name',
 'category',
 'main_category',
 'currency',
 'deadline',
 'goal',
 'launched',
 'pledged',
 'state',
 'backers',
 'country',
 'usd pledged',
 'usd_pledged_real',
 'usd_goal_real']

In [31]:
df['currency'].value_counts()

USD    295365
GBP     34132
EUR     17405
CAD     14962
AUD      7950
SEK      1788
MXN      1752
NZD      1475
DKK      1129
CHF       768
NOK       722
HKD       618
SGD       555
JPY        40
Name: currency, dtype: int64

In [ ]:
df['deadline'].sort_values(ascending=False)

In [32]:
df['main_category'].value_counts()

Film & Video    63585
Music           51918
Publishing      39874
Games           35231
Technology      32569
Design          30070
Art             28153
Food            24602
Fashion         22816
Theater         10913
Comics          10819
Photography     10779
Crafts           8809
Journalism       4755
Dance            3768
Name: main_category, dtype: int64

In [34]:
state_grp = df.groupby('state')

In [35]:
state_grp.size()

state
canceled       38779
failed        197719
live            2799
successful    133956
suspended       1846
undefined       3562
dtype: int64

In [40]:
state_grp['main_category'].value_counts()

state       main_category
canceled    Games             6202
            Film & Video      5755
            Technology        4715
            Design            4152
            Publishing        3602
            Music             3305
            Fashion           2650
            Art               2222
            Food              2211
            Photography        986
            Crafts             843
            Comics             842
            Theater            608
            Journalism         523
            Dance              163
failed      Film & Video     32904
            Publishing       23145
            Music            21752
            Technology       20616
            Games            16003
            Food             15969
            Design           14814
            Fashion          14182
            Art              14131
            Photography       6384
            Crafts            5703
            Comics            4036
            Theater          

In [38]:
state_grp.agg('count')

state
canceled       38779
failed        197719
live            2799
successful    133956
suspended       1846
undefined       3562
Name: main_category, dtype: int64

In [52]:
df_suc_fail = df.loc['successful']

KeyError: 'successful'